In [ ]:
%matplotlib nbagg
import numpy as np
import scipy.io
import scipy.ndimage
import scipy.signal
import matplotlib.pyplot as plt
import matplotlib.animation
import cmasher as cmr
import os

folder=r"J:\ctgroup\Edward\DATA\VMI\20250303\Propylene Oxide 2W"
folder=r"J:\ctgroup\Edward\DATA\VMI\20250227\xe_2,5W"
files=[os.path.join(folder,f) for f in os.listdir(folder) if f.endswith('.mat')]
angles=[float(f.split("\\")[-1].split('_')[0]) for f in files]
angles,files=zip(*sorted(zip(angles,files)))



In [ ]:
def load_file(file, smoothed=True, plot=False, py_shift=0, angle=0):
    data=scipy.io.loadmat(file,squeeze_me=True)
    px,py,pz=data['px'],data['py']+py_shift,data['pz']
    h=np.histogram(np.sqrt(px**2+py**2+pz**2),bins=500,range=(0,1))

    hist,edges=h[0],h[1]
    if smoothed:
        hist=scipy.signal.savgol_filter(hist,51,3)
    hist=hist/np.max(hist)

    peaks,properties=scipy.signal.find_peaks(hist,prominence=0.07)
    peak_widths=scipy.signal.peak_widths(hist,peaks,rel_height=0.2)[0]

    ring_vals=np.array([edges[int(peaks[i])] for i in range(len(peaks))])
    ring_widths=np.array([edges[int(peak_widths[i])] for i in range(len(peaks))])

    if plot:
        plt.figure(file)
        plt.plot(edges[:-1],hist)
        plt.plot(edges[peaks],hist[peaks],'x')
        [plt.plot([ring_vals[i]-ring_widths[i]*2,ring_vals[i]+ring_widths[i]*2],[0.5,0.5]) for i in range(len(ring_vals))]
    pr=np.sqrt(px**2+py**2+pz**2)
    masks=[(pr>ring_vals[i]-ring_widths[i]*2) & (pr<ring_vals[i]+ring_widths[i]*2) for i in range(len(ring_vals))]
    reasonable=np.abs(pz)<np.max(px)

    rings=[(px[masks[i]],py[masks[i]],pz[masks[i]]) for i in range(len(ring_vals))]
    px,pz= px*np.cos(angle)+pz*np.sin(angle),-px*np.sin(angle)+pz*np.cos(angle)
    return {
        'px':px[reasonable],
        'py':py[reasonable],
        'pz':pz[reasonable],
        'rings':rings,
        'ring_vals':ring_vals,
        'ring_widths':ring_widths
    }
data=[load_file(f,plot=False) for f in files]

In [ ]:
n=256
thetav,phiv=np.linspace(0,np.pi,n),np.linspace(-np.pi,np.pi,n*2)
theta,phi=np.meshgrid(thetav,phiv)
dtheta,dphi=np.pi/n,2*np.pi/n/2
domega=np.sin(theta)*dtheta*dphi
lmax=19
Y={(l,m): scipy.special.sph_harm(m,l,phi,theta) for l in range(lmax+1) for m in range(-l,l+1)}
Y_real={(l,m): np.real(
        int(m<0)*1j*np.sqrt(1/2)*(Y[(l,m)]-Y[(l,-m)]*(-1)**m)+
        int(m==0)*Y[(l,0)]+
        int(m>0)*np.sqrt(1/2)*(Y[(l,m)]+Y[(l,-m)]*(-1)**m)

)
    for l in range(lmax+1) for m in range(-l,l+1)
}

def get_beta(data,ring_number,Y,lmax,smoothing=3, symmetrize=False, n=1, k=0, angle=0, flip=False):
    ring=data['rings'][ring_number]
    px,py,pz=ring
    px,py,pz=px[k::n],py[k::n],pz[k::n]
    px,pz=px*np.cos(angle)+pz*np.sin(angle),-px*np.sin(angle)+pz*np.cos(angle)
    if flip:
        px,pz=-px,-pz
    if symmetrize:
        px,py,pz=np.vstack([px,-px]),np.vstack([py,py]),np.vstack([pz,-pz])
        px,py,pz=px.flatten(),py.flatten(),pz.flatten()

    pr=np.sqrt(px**2+py**2+pz**2)
    ptheta=np.arccos(py/pr)
    pphi=np.arctan2(px,pz)

    hist=np.histogram2d(pphi,ptheta,bins=(len(phiv),len(thetav)),density=True)[0]
    if smoothing>0:
        hist=scipy.ndimage.gaussian_filter(hist,smoothing)
    beta={(l,m): np.sum(hist*Y[(l,m)]*domega) for l in range(lmax+1) for m in range(-l,l+1)}
    return beta

betas={(angle,ring): get_beta(datum,ring,Y_real,lmax, angle=-angle, symmetrize=True) for angle,datum in zip(angles,data) for ring in range(len(datum['rings']))}

In [ ]:
# %matplotlib inline

import plotly.express as px
import pandas as pd

# plt.figure(figsize=(15, 10))

# angle=45
ring=0
lmax=19
betas_df=pd.DataFrame()
for angle in angles:
    beta=betas[(angle,ring)]
    for l in range(lmax+1):
        for m in range(-l,l+1):
            es= l%2==1
            dichroic= (l%2==1 and m>=0) or (l%2==0 and m<0)
            type=""
            type+= "Dichroic" if dichroic else ""
            type+= " and " if dichroic and es else ""
            type+= "Enantiosensitive" if es else ""
            if not type:
                type="Neither"
            
            row={
                'angle':angle,
                'helicity': angle>0,
                'ellipticity': np.tan(np.radians(angle-1.7)),
                'l':l,
                'm':m,
                'lg': (l, m, type),
                'beta':beta[(l,m)],
                "allowed": m%2==0,
                'enantiosensitive': es,
                'dichroic': dichroic,
                'type': type
            }
            betas_df = pd.concat([betas_df,pd.DataFrame([row])])

In [ ]:
lmax=5
angle=40
ellipticity=np.tan(np.radians(angle))
betas_df_filtered=betas_df[(betas_df['l']<=lmax) & (abs(betas_df['angle'])==angle)]
# plt.figure(figsize=(10, 10))

# plt.subplot(321)
# angle_idx=angles.index(-angle)
# pxr,pzr=data[angle_idx]['px']*np.cos(angle)+data[angle_idx]['pz']*np.sin(angle),-data[angle_idx]['px']*np.sin(angle)+data[angle_idx]['pz']*np.cos(angle)
# plt.hist2d(pxr,pzr,bins=512, cmap=cmr.rainforest, range=[[-0.5,0.5],[-0.5,0.5]])
# plt.axis('off')
# plt.axis('equal')
# plt.subplot(322)
# angle_idx=angles.index(angle)
# pxr,pzr=data[angle_idx]['px']*np.cos(-angle)+data[angle_idx]['pz']*np.sin(-angle),-data[angle_idx]['px']*np.sin(-angle)+data[angle_idx]['pz']*np.cos(-angle)
# plt.hist2d(pxr,pzr,bins=512, cmap=cmr.rainforest, range=[[-0.5,0.5],[-0.5,0.5]])
# plt.axis('off')
# plt.axis('equal')
# 
# 
# def reconstruct(betas,lmax,angle,ring, lmin=0):
#     reconstruction= np.sum([betas[(angle,ring)][(l,m)]*Y_real[(l,m)] for l in range(lmin,lmax+1) for m in range(-l,l+1)],axis=0)
#     return reconstruction
# 
# plt.subplot(312)
# reconstruction=reconstruct(betas,12,angle,ring, lmin=2)
# plt.imshow(reconstruction.T, cmap=cmr.ember, extent=[-np.pi,np.pi,-np.pi/2,np.pi/2])
# 
# plt.subplot(313)
# reconstruction=reconstruct(betas,12,-angle,ring, lmin=2)
# plt.imshow(reconstruction.T, cmap=cmr.ember, extent=[-np.pi,np.pi,-np.pi/2,np.pi/2])
# 
# plt.tight_layout()
# display(betas_df[(abs(betas_df['angle'])==angle) & (betas_df['l']==2)])
# load_file(files[angles.index(angle)],plot=True)
# load_file(files[angles.index(-angle)],plot=True)


fig=px.scatter(betas_df_filtered,x='m',y='beta',color='helicity',symbol='dichroic',facet_col='l',color_discrete_sequence=['red','blue'], symbol_map={'True':'circle','False':'x'}, facet_col_wrap=3,title=f"Ellipticity: {ellipticity:.2f}")
# fig.update_yaxes(type="log")
fig.update_layout({
    "height": 600,
    "width": 1200,
})
fig.show()

In [ ]:
lmax=6
px.line(betas_df[(betas_df['l']<lmax)&betas_df['allowed']], 
           x='ellipticity', y='beta',
           line_group='lg',
           color='l', symbol='m', 
           facet_col='type', facet_col_wrap=2, 
           title="Beta Coefficients vs Ellipticity (Symmetry Allowed)",
           width=1600, height=800).show()

In [ ]:

beta_diffs={l:betas[(angle,ring)][(l,m)]-(-1)**m*betas[(-angle,ring)][(l,m)] for l in range(lmax+1) for m in range(-l,l+1) for angle in angles}

beta_diffs_df=pd.DataFrame()



In [ ]:



for angle in [-35,35]:
    beta=betas[(angle,ring)]
    plt.suptitle(f"Angle: {angle}, Ring: {ring+1}, All Beta Coefficients")
    plt.subplot(311)
    beta_magnitudes=[np.sum([beta[(l,m)]**2 for m in range(-l,l+1)]) for l in range(lmax+1)]
    beta_magnitudes=np.array(beta_magnitudes)/np.max(beta_magnitudes)
    plt.xticks([i for i in range(lmax+1)])
    plt.semilogy(beta_magnitudes)
    plt.xlabel('l')
    plt.ylabel(r"$\sum_m \beta_{lm}^2$")
    plt.grid()

    plt.subplot(334)
    plt.plot(range(-1,2),[beta[(1,m)] for m in range(-1,2)])
    plt.title("l=1")
    plt.xticks([-1,0,1])
    plt.grid()

    plt.subplot(335)
    plt.plot(range(-2,3),[beta[(2,m)] for m in range(-2,3)])
    plt.title("l=2")
    plt.xticks([-2,-1,0,1,2])
    plt.grid()

    plt.subplot(336)
    plt.plot(range(-3,4),[beta[(3,m)] for m in range(-3,4)])
    plt.title("l=3")
    plt.xticks([-3,-2,-1,0,1,2,3])
    plt.grid()

    plt.subplot(337)
    plt.plot(range(-4,5),[beta[(4,m)] for m in range(-4,5)])
    plt.title("l=4")
    plt.xticks([-4,-3,-2,-1,0,1,2,3,4])
    plt.grid()

    plt.subplot(338)
    plt.plot(range(-5,6),[beta[(5,m)] for m in range(-5,6)])
    plt.title("l=5")
    plt.xticks([-5,-4,-3,-2,-1,0,1,2,3,4,5])
    plt.grid()

    plt.subplot(339)
    plt.plot(range(-6,7),[beta[(6,m)] for m in range(-6,7)])
    plt.title("l=6")
    plt.xticks([-6,-5,-4,-3,-2,-1,0,1,2,3,4,5,6])
    plt.grid()

plt.tight_layout()
plt.show()